In [1]:
# Dependencies
import os
import pandas as pd
from citipy import citipy
import numpy as np
from pprint import pprint
from config import api_key
import matplotlib.pyplot as plt
import json
import requests

## Create a list of cities

In [2]:
# Create a random list of latitudes and longitudes
# np.random.uniform makes sure that all the numbers in the range get equal chance of getting picked

latitudes = np.random.uniform(-90, 90, size = 2000) # 500 random numbers from -90 to 90 deg latitude
longitudes = np.random.uniform(-180, 180, size = 2000) # 500 random numbers from -180 to 180 deg longitude

coordinates = list(zip(latitudes, longitudes))

In [3]:
# Nearby cities per latitude-longitude pair
cities = []

for coord in coordinates:
    lat, lon = coord
    cities.append(citipy.nearest_city(lat, lon))

In [4]:
# Set of cities
city_names = []
country = []

for city in cities:
    city_names.append(city.city_name) # loop through the city coordinates to get the city names
    country.append(city.country_code) # loop through the city coordinates to get the city names
    
city_dict ={
    "city": city_names,
    "country": country,
    "latitude": latitudes,
    "longitude": longitudes
           }    

city_df = pd.DataFrame(city_dict)
city_df = city_df.drop_duplicates(["city","country"])
city_df = city_df.dropna()
len(city_df)

770

In [5]:
# Add new columns
city_df["Temperature (F)"] = ""
city_df["Humidity (%)"] = ""
city_df["Cloudiness (%)"] = ""
city_df["Wind Speed (mph)"] = ""
city_df.head()

,city,country,latitude,longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,sopelana,es,43.731374,-3.000585,,,,
1,ushuaia,ar,-68.986720,-53.435644,,,,
2,banda aceh,id,2.242317,90.315992,,,,
3,grand gaube,mu,-15.505464,64.641149,,,,
4,tasiilaq,gl,76.497003,-34.247752,,,,


In [6]:
# Get JSON file containing city ID
filepath = os.path.join("Resources","city.list.json")
with open(filepath) as json_file:
    json_data = json.load(json_file)

In [7]:
# Get the city name, country code, and city ID
ct_name = []
co_name = []
ct_ID = []

for i in json_data:
    ct_name.append(i["name"]) # city name from JSON file
    co_name.append(i["country"]) # country name from JSON file
    ct_ID.append(i["id"]) # city ID from JSON file

In [8]:
# Put the JSON-sourced data into a dataframe
json_dict = {
    "city": ct_name,
    "country": co_name,
    "city ID": ct_ID}

json_df = pd.DataFrame(json_dict)
json_df["city"] = json_df["city"].str.lower()
json_df["country"] = json_df["country"].str.lower()
json_df.head()

,city,country,city ID
0,hurzuf,ua,707860
1,novinki,ru,519188
2,gorkhā,np,1283378
3,state of haryāna,in,1270260
4,holubynka,ua,708546


In [18]:
# Merge json_df and city_df
city_df2 = pd.merge(city_df,json_df, on = ["city", "country"])
city_df2 = city_df2.drop_duplicates(["city", "country"])
print(f"There are {len(city_df2)} cities in this dataframe.")
city_df2.head()

There are 654 cities in this dataframe.


,city,country,latitude,longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),city ID
0,sopelana,es,43.731374,-3.000585,,,,,6362425
2,ushuaia,ar,-68.986720,-53.435644,,,,,3833367
3,banda aceh,id,2.242317,90.315992,,,,,1215501
5,grand gaube,mu,-15.505464,64.641149,,,,,934479
6,tasiilaq,gl,76.497003,-34.247752,,,,,3424607


## Get the weather for each city through the OpenWeatherData API

In [ ]:
# Formulate the query URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Loop through the rows to get the different cities and countries
for index, row in city_df2.iterrows():
    x = row["city"]
    y = row["country"]
    
    # Create a variable for iteration through the dataframe
    city_ID = row["city ID"]
    
    # Create a query URL
    query_url = f"{url}&appid={api_key}&id={city_ID}&units={units}"
    
    # Create a request
    #print(f"Retrieving information for Index {index}: {city_ID}.")
    response = requests.get(query_url)
    response_json = response.json()
pprint(response_json)
    
#   # Request and JSON-ify the response 
#     
#     
# 
#     # Extract response
#     try:
#         print(f"{x},{y} has {response_json['clouds']['all']}.")
        
#         city_df.loc[index, "Cloudiness (%)"] = response_json["clouds"]["all"]
#         city_df.loc[index, "Humidity (%)"] = response_json["main"]["humidity"]
#         city_df.loc[index, "Wind Speed (mph)"] = response_json["wind"]["speed"]
#         city_df.loc[index, "Temperature (F)"] = response_json["main"]["temp"]
    
#     except (KeyError, IndexError):
#         print("City is not on the list. Next!")
    
#     print("----------")

In [ ]:
#city_df.head()